In [1]:
from PIL import Image
import os, glob
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
target_size=(28,28)
for path in ['rock', 'scissor', 'paper']:
    image_path = image_dir_path + '/' + path
    images=glob.glob(image_path + "/*.jpg")  
    print(f'len({path}) = ', len(images))
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img,"JPEG")

len(rock) =  100
len(scissor) =  100
len(paper) =  100


In [6]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)


In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_channel_1=128
n_channel_2=256
n_dense= 256
n_train_epoch= 10
img_size = 28

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(img_size,img_size,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 128)       3584      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 256)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               1638656   
_________________________________________________________________
dense_5 (Dense)              (None, 3)                

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=n_train_epoch)

Epoch 1/10
10/10 [==============================] - 3s 268ms/step - loss: 62.3813 - accuracy: 0.3100
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: 1.2977 - accuracy: 0.3833
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: 0.9487 - accuracy: 0.5833
Epoch 4/10
10/10 [==============================] - 0s 3ms/step - loss: 0.6075 - accuracy: 0.6900
Epoch 5/10
10/10 [==============================] - 0s 3ms/step - loss: 0.3663 - accuracy: 0.8400
Epoch 6/10
10/10 [==============================] - 0s 3ms/step - loss: 0.2289 - accuracy: 0.9300
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 0.1260 - accuracy: 0.9667
Epoch 8/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0719 - accuracy: 0.9833
Epoch 9/10
10/10 [==============================] - 0s 2ms/step - loss: 0.0702 - accuracy: 0.9900
Epoch 10/10
10/10 [==============================] - 0s 3ms/step - loss: 0.0482 - accuracy: 0.9967


In [11]:
# 사이즈 변경시에만 사용하기
target_size=(img_size, img_size)
for path in ['t_rock', 't_scissor', 't_paper']:
    image_path = image_dir_path + '/' + path
    images=glob.glob(image_path + "/*.jpg")  
    print(f'len({path}) = ', len(images))
    for img in images:
        old_img=Image.open(img)
        # print('size = ', old_img.size)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img,"JPEG")

len(t_rock) =  50
len(t_scissor) =  50
len(t_paper) =  50


In [12]:
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=150   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)
    idx=0
    for file in glob.iglob(img_path+'/t_scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
    for file in glob.iglob(img_path+'/t_rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1
    for file in glob.iglob(img_path+'/t_paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1

    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 150 입니다.
x_test shape: (150, 28, 28, 3)
y_test shape: (150,)


In [13]:
test_loss, test_accuracy = model.evaluate(x_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

5/5 - 1s - loss: 0.8787 - accuracy: 0.8067
test_loss: 0.8787221312522888 
test_accuracy: 0.8066666722297668
